# Notebook One

This notebook takes the three separate formats of the published Allegheny parallaxes and formats them all into Astropy tables matched with their YPC counterparts.

In [1]:
%pylab inline
from astropy.table import Table, hstack,vstack, Column, QTable
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK5
from astropy.time import Time
from astropy.coordinates import Angle
from astropy.io import ascii



# Astropy
from astropy import coordinates as coord
from astropy import units as u

# Astroquery. 
import astroquery
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.vo_conesearch import conf, conesearch, vos_catalog

Populating the interactive namespace from numpy and matplotlib


## Read in Raw Data

In [2]:

table1 = QTable.read('1_365.csv')

print(table1)

﻿Number       Name       RA 1900 (Hour) ... Error Error for one good plate
------- ---------------- -------------- ... ----- ------------------------
      1 Sigma Andromedae              0 ... 0.008                    0.022
      2    Delta Piscium              0 ...   7.0                     20.0
      3  Epsilon Piscium              0 ...   8.0                     23.0
      4   Mu Cassiopeiae              1 ...   9.0                     23.0
      5 Omicron  Piscium              1 ...   6.0                     18.0
      6     Beta Arietis              1 ...   8.0                     19.0
      7       39 Arietis              2 ...   6.0                     16.0
      8         37 Tauri              3 ...   9.0                     24.0
      9        50 Persei              4 ...   8.0                     22.0
     10     Mu Geminorum              6 ...   8.0                     20.0
    ...              ...            ... ...   ...                      ...
    355 Groombridge 3243 

In [3]:
print(table1.colnames)

['\ufeffNumber', 'Name', 'RA 1900 (Hour)', 'RA 1900 (Min)', 'Dec 1900 (Degrees)', 'Dec 1900 (Min)', 'D Number 1', 'D Number 2', 'Mag.', 'Spectral type', 'Prop. Mot.', 'Rel. Parallax', 'Error', 'Error for one good plate']


In [4]:
np.ma.fix_invalid(table1['RA 1900 (Hour)'],fill_value=np.nan) # Reads empty values in the raw table as "masked" values,
                                                              # so find all of those and replace with something uniform, like np.nan



0
0
0
1
1
1
2
3
4
6
6


In [5]:
table2 = QTable.read('366_779.csv')
table2.show_in_notebook()

idx,﻿Number,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),D Number 1,D Number 2,Mag.,Spectral Type,P.M. Catalog,P.M. AO,Rel. Parallax,P.E,P.E. Wt,sqrt(p pi),No. Plates
0,366,22 Andr,0,5.1,45,31,45,17,5.1,FO,0.006,0.011,-0.01,0.01,0.025,2.5,14
1,367,Lal 617,0,23.2,9,39,9,47,6,F2,0.031*,0.031,0.029,7.0,21.0,3.0,17
2,368,Phi Pisc,1,8.3,24,3,23,158,4.6,KO,0.025,-0.032,-0.01,8.0,23.0,2.9,15
3,369,Omega Andr.,1,21.7,44,53,44,307,5,F5,0.346,0.337,0.024,4.0,13.0,3.0,15
4,370,Piazzi 145,1,35.7,25,14,25,276,6.3,F5,0.13,0.106,0.009,10.0,24.0,2.5,14
5,371,Lal 4268,2,12.8,1,17,1,410,5.8,F8,0.371,0.4,0.043,10.0,25.0,2.6,12
6,372,Lal 5376,2,49.7,26,28,26,484,7.4,G5,0.258*,0.269,0.056,7.0,20.0,2.8,14
7,373,Mayer 120,3,28.4,17,30,17,575,6.4,KO,0.091*,0.083,0.021,8.0,23.0,3.1,15
8,374,Lal 7036,3,46.4,60,53,60,762,7.8,KO,0.418*,0.385,0.048,8.0,20.0,2.7,14
9,375,Omega Sigma 531,4,0.9,37,49,37,878,7.1,G5,0.183,0.2,0.026,7.0,22.0,3.0,18


In [6]:
np.ma.fix_invalid(table2['RA 1900 (Hour)'],fill_value=np.nan)


0
0
1
1
1
2
2
3
3
4
4


In [7]:
table3 = QTable.read('780_2070.csv')
print(table3)

﻿No.    Name    RA 1900 (Hour) RA 1900 (Min) ...  P.E.  Ptg. Mag. No. Pl
---- ---------- -------------- ------------- ... ------ --------- ------
 780    38 Ceti              1           9.7 ...  7; 24      11.6     21
 781 Theta Ceti              1          19.0 ...  5; 18      11.9     21
 782   107 Pisc              1          37.1 ...  9; 30      12.1     19
 783     6 Tria              2           6.6 ...  6; 23      11.8     23
 784     7 Taur              3          28.5 ...  5; 17      11.2     21
 785  Lal 11218              5          51.3 ...  6; 22      10.6     20
 786  Lal 15743              7          58.8 ...  5; 15      11.2     18
 787    14 Boot             14           9.3 ...  8; 27      10.3     20
 788      Pi 69             14          18.5 ...  7; 23      11.4     21
 789  Lal 26635             14          31.7 ...  7; 22      10.8     19
 ...        ...            ...           ... ...    ...       ...    ...
2060    WB 1259             16          41.5 ...   

In [8]:
np.ma.fix_invalid(table3['RA 1900 (Hour)'],fill_value=np.nan)

1
1
1
2
3
5
7
14
14
14
16


### This section is no longer relevant
It is used to precess coordinates to J2000 given any starting equinox. The default is set as J1900

In [9]:
def convert_to_J200(ra_h,ra_min,dec_d,dec_min,start_equinox='J1900'):
    """
    Input Parameters:Give function RA hour, min, seconds,
                                   Dec deg, min, seconds
                                   Starting equinox: default = 1900
    Returns: RA in format: 00h00m00.00s
             Dec in format: 00d00m00.00s
    """
    
    ra_minute = str(ra_min)
    ra_minute = ra_minute[:2]
    ra_sec = "%.2f" % ((ra_min * 60) % 60)
    
    dec_sec = "%.2f" % ((abs(dec_min) * 60) % 60)
 
    dec_d = str(dec_d)
    
    if dec_d[0] != '-':
        dec_d = '+' + dec_d
        
    if np.isnan(ra_h) == False:
        ra = str(ra_h) + 'h' + str(ra_minute) + 'm' + str(ra_sec) + 's'
        dec = (dec_d) + 'd' + str(dec_min) + 'm' + str(dec_sec) + 's'
    

    
        fk0c = SkyCoord(ra, dec, frame=FK5(equinox=Time(start_equinox)))
        fk0_2000 = FK5(equinox='J2000')
        c = fk0c.transform_to(fk0_2000,type)
        string = c.to_string('hmsdms')
        #print(string)
        ra2000,dec2000 = string.split()[0], string.split()[1]
        
        
    else:
        ra2000,dec2000 = 0,0
        c = 0 

    
    return(c,ra2000,dec2000)
    #return c

In [10]:
c,r,d = convert_to_J200(0,13,36,14)
print(c,r,d)

<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (4.55885143, 36.78886823)> 00h18m14.1243s +36d47m19.9256s


### This section is no longer relevant
#### Conesearch:
Used an astropy builtin function to search for stars within a "cone" of sky. Given the celestial coordinates, this function will search in a 0.02 (current default) degree cone of the sky. 


In [11]:
def search_coord(ra_h,ra_min,dec_d,dec_min):
    c,ra, dec = convert_to_J200(ra_h,ra_min,dec_d,dec_min)
    result = conesearch.conesearch(c, 0.02 * u.degree)
    ra_colname = 'ra'
    dec_colname = 'dec'
    
    if ra_h == 0:
        return 0,0,0
    
    elif result is None:
    #print('{} or {} not in search results. Choose from: {}'.format(
        #ra_colname, dec_colname, ' '.join(result.colnames)))
        return 0,0,0

    elif ra_colname in result.colnames and dec_colname in result.colnames:
        row = result[0]
        simbad_obj = coord.SkyCoord(ra=row[ra_colname]*u.deg, dec=row[dec_colname]*u.deg)
        #print('Searching SIMBAD for\n{}\n'.format(simbad_obj))
        simbad_result = Simbad.query_region(simbad_obj, radius=5*u.arcmin)
        if simbad_result is None:
            return 0,0,0
        else:
            return (simbad_result['MAIN_ID'][0],simbad_result['RA'][0],simbad_result['DEC'][0])


In [12]:
#name, ra, dec = search_coord(0,13,36,14)
#name,ra,dec = search_coord(6,17,22,34)
print(table1[14])
name,ra,dec = search_coord(16,11,34,7)
print(name,ra,dec)


﻿Number         Name         RA 1900 (Hour) RA 1900 (Min) Dec 1900 (Degrees) Dec 1900 (Min) D Number 1 D Number 2 Mag. Spectral type Prop. Mot. Rel. Parallax Error Error for one good plate
------- -------------------- -------------- ------------- ------------------ -------------- ---------- ---------- ---- ------------- ---------- ------------- ----- ------------------------
     15 Sigma  Coronae (fol)             --            --                 --             --         --         --  5.8            --         --         0.052  10.0                     21.0
Trying http://gsss.stsci.edu/webservices/vo/ConeSearch.aspx?CAT=GSC23&
b'TYC 2583-1900-1' 16 14 47.9546 +33 51 42.651


### This section is no longer relevant
Added columns for J2000 coordinates, precessed from published 1900 ones and then attempted to use the cone search function with precessed J2000 coordinates

###  Table 1 (0-365):

In [13]:
RA2000 = Column(np.arange(len(table1)), name='RA 2000')
table1.add_column(RA2000, index=6)
table1['RA 2000'] = table1['RA 2000'].astype(str)

Dec2000 = Column(np.arange(len(table1)), name='Dec 2000')
table1.add_column(Dec2000, index=7)
table1['Dec 2000'] = table1['Dec 2000'].astype(str)

table1['Error'] = table1['Error'].astype(str)
table1['Rel. Parallax'] = table1['Rel. Parallax'].astype(str)
table1['Error for one good plate'] = table1['Error for one good plate'].astype(str)

"""
query_name = Column(np.arange(len(table1)), name='Query Name')
table1.add_column(query_name, index=8)
table1['Query Name'] = table1['Query Name'].astype(str)

query_ra_2000 = Column(np.arange(len(table1)), name='Query RA 2000')
table1.add_column(query_ra_2000, index=9)
table1['Query RA 2000'] = table1['Query RA 2000'].astype(str)

query_dec_2000 = Column(np.arange(len(table1)), name='Query Dec 2000')
table1.add_column(query_dec_2000, index=10)
table1['Query Dec 2000'] = table1['Query Dec 2000'].astype(str)


"""



"\nquery_name = Column(np.arange(len(table1)), name='Query Name')\ntable1.add_column(query_name, index=8)\ntable1['Query Name'] = table1['Query Name'].astype(str)\n\nquery_ra_2000 = Column(np.arange(len(table1)), name='Query RA 2000')\ntable1.add_column(query_ra_2000, index=9)\ntable1['Query RA 2000'] = table1['Query RA 2000'].astype(str)\n\nquery_dec_2000 = Column(np.arange(len(table1)), name='Query Dec 2000')\ntable1.add_column(query_dec_2000, index=10)\ntable1['Query Dec 2000'] = table1['Query Dec 2000'].astype(str)\n\n\n"

In [14]:
table1.colnames

['\ufeffNumber',
 'Name',
 'RA 1900 (Hour)',
 'RA 1900 (Min)',
 'Dec 1900 (Degrees)',
 'Dec 1900 (Min)',
 'RA 2000',
 'Dec 2000',
 'D Number 1',
 'D Number 2',
 'Mag.',
 'Spectral type',
 'Prop. Mot.',
 'Rel. Parallax',
 'Error',
 'Error for one good plate']

In [15]:
## add columns 


for i in range(len(table1)):
    ra_h, ra_min, dec_d, dec_min = table1[i]['RA 1900 (Hour)'],table1[i]['RA 1900 (Min)'].astype(int),table1[i]['Dec 1900 (Degrees)'],table1[i]['Dec 1900 (Min)']
    c,ra, dec = convert_to_J200(ra_h,ra_min,dec_d,dec_min)
    
    #name, ra_, dec_ = search_coord(ra_h, ra_min, dec_d, dec_min)
    
    #print(i)
    table1[i]['RA 2000'] = ra
    table1[i]['Dec 2000'] = dec
    #table1[i]['Query Name'] = name
    #table1[i]['Query RA 2000'] = ra_
    #table1[i]['Query Dec 2000'] = dec_
    
    #table1[i]['c 2000'] = c

    
    
    
    
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/a

### Table 2 (366-779):

In [16]:
table2.colnames

['\ufeffNumber',
 'Name',
 'RA 1900 (Hour)',
 'RA 1900 (Min)',
 'Dec 1900 (Degrees)',
 'Dec 1900 (Min)',
 'D Number 1',
 'D Number 2',
 'Mag.',
 'Spectral Type',
 'P.M. Catalog',
 'P.M. AO',
 'Rel. Parallax',
 'P.E',
 'P.E. Wt',
 'sqrt(p pi)',
 'No. Plates']

In [17]:
RA2000_t2 = Column(np.arange(len(table2)), name='RA 2000')
table2.add_column(RA2000_t2, index=6)
table2['RA 2000'] = table2['RA 2000'].astype(str)

Dec2000_t2 = Column(np.arange(len(table2)), name='Dec 2000')
table2.add_column(Dec2000_t2, index=7)
table2['Dec 2000'] = table2['Dec 2000'].astype(str)

table2['Rel. Parallax'] = table2['Rel. Parallax'].astype(str)
table2['P.E'] = table2['P.E'].astype(str)
table2['P.E. Wt'] = table2['P.E. Wt'].astype(str)


"""
query_name_t2 = Column(np.arange(len(table2)), name='Query Name')
table2.add_column(query_name_t2, index=8)
table2['Query Name'] = table2['Query Name'].astype(str)

query_ra_2000_t2 = Column(np.arange(len(table2)), name='Query RA 2000')
table2.add_column(query_ra_2000_t2, index=9)
table2['Query RA 2000'] = table2['Query RA 2000'].astype(str)

query_dec_2000_t2 = Column(np.arange(len(table2)), name='Query Dec 2000')
table2.add_column(query_dec_2000_t2, index=10)
table2['Query Dec 2000'] = table2['Query Dec 2000'].astype(str)
"""


"\nquery_name_t2 = Column(np.arange(len(table2)), name='Query Name')\ntable2.add_column(query_name_t2, index=8)\ntable2['Query Name'] = table2['Query Name'].astype(str)\n\nquery_ra_2000_t2 = Column(np.arange(len(table2)), name='Query RA 2000')\ntable2.add_column(query_ra_2000_t2, index=9)\ntable2['Query RA 2000'] = table2['Query RA 2000'].astype(str)\n\nquery_dec_2000_t2 = Column(np.arange(len(table2)), name='Query Dec 2000')\ntable2.add_column(query_dec_2000_t2, index=10)\ntable2['Query Dec 2000'] = table2['Query Dec 2000'].astype(str)\n"

In [18]:
#fill columns

for i in range(len(table2)):
    ra_h, ra_min, dec_d, dec_min = table2[i]['RA 1900 (Hour)'],table2[i]['RA 1900 (Min)'].astype(int),table2[i]['Dec 1900 (Degrees)'],table2[i]['Dec 1900 (Min)']
    c,ra, dec = convert_to_J200(ra_h,ra_min,dec_d,dec_min)
    
    #name, ra_, dec_ = search_coord(ra_h, ra_min, dec_d, dec_min)
    
    table2[i]['RA 2000'] = ra
    table2[i]['Dec 2000'] = dec
    #table2[i]['Query Name'] = name
    #table2[i]['Query RA 2000'] = ra_
    #table2[i]['Query Dec 2000'] = dec_


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/a

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  


### Table 3 (780 - 2045):

In [19]:
table3.colnames

['\ufeffNo.',
 'Name',
 'RA 1900 (Hour)',
 'RA 1900 (Min)',
 'Dec 1900 (Degrees)',
 'Dec 1900 (Min)',
 'D. Number 1',
 'D Number 2',
 'Mag.',
 'Spectral Type',
 'Tot. Mu',
 'Mu alpha',
 'AO Mu alpha',
 'Rel. Parallax',
 'P.E.',
 'Ptg. Mag.',
 'No. Pl']

In [20]:
RA2000_t3 = Column(np.arange(len(table3)), name='RA 2000')
table3.add_column(RA2000_t3, index=6)
table3['RA 2000'] = table3['RA 2000'].astype(str)

Dec2000_t3 = Column(np.arange(len(table3)), name='Dec 2000')
table3.add_column(Dec2000_t3, index=7)
table3['Dec 2000'] = table3['Dec 2000'].astype(str)

table3['Rel. Parallax'] = table3['Rel. Parallax'].astype(str)
table3['P.E.'] = table3['P.E.'].astype(str)

"""
query_name_t3 = Column(np.arange(len(table3)), name='Query Name')
table3.add_column(query_name_t3, index=8)
table3['Query Name'] = table3['Query Name'].astype(str)

query_ra_2000_t3 = Column(np.arange(len(table3)), name='Query RA 2000')
table3.add_column(query_ra_2000_t3, index=9)
table3['Query RA 2000'] = table3['Query RA 2000'].astype(str)

query_dec_2000_t3 = Column(np.arange(len(table3)), name='Query Dec 2000')
table3.add_column(query_dec_2000_t3, index=10)
table3['Query Dec 2000'] = table3['Query Dec 2000'].astype(str)
"""

"\nquery_name_t3 = Column(np.arange(len(table3)), name='Query Name')\ntable3.add_column(query_name_t3, index=8)\ntable3['Query Name'] = table3['Query Name'].astype(str)\n\nquery_ra_2000_t3 = Column(np.arange(len(table3)), name='Query RA 2000')\ntable3.add_column(query_ra_2000_t3, index=9)\ntable3['Query RA 2000'] = table3['Query RA 2000'].astype(str)\n\nquery_dec_2000_t3 = Column(np.arange(len(table3)), name='Query Dec 2000')\ntable3.add_column(query_dec_2000_t3, index=10)\ntable3['Query Dec 2000'] = table3['Query Dec 2000'].astype(str)\n"

In [21]:
for i in range(len(table3)):
    print(i)
    ra_h, ra_min, dec_d, dec_min = table3[i]['RA 1900 (Hour)'],table3[i]['RA 1900 (Min)'].astype(int),table3[i]['Dec 1900 (Degrees)'],table3[i]['Dec 1900 (Min)']
    c,ra, dec = convert_to_J200(ra_h,ra_min,dec_d,dec_min)
    
    #name, ra_, dec_ = search_coord(ra_h, ra_min, dec_d, dec_min)
    table3[i]['RA 2000'] = ra
    table3[i]['Dec 2000'] = dec
    #table3[i]['Query Name'] = name
    #table3[i]['Query RA 2000'] = ra_
    #table4[i]['Query Dec 2000'] = dec_

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/a


65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
32

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/a

...

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/a


569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/a


704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':
/opt/a

In [22]:
table3[1311]

﻿No.,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D. Number 1,D Number 2,Mag.,Spectral Type,Tot. Mu,Mu alpha,AO Mu alpha,Rel. Parallax,P.E.,Ptg. Mag.,No. Pl
str5,str44,int64,float64,int64,int64,str21,str21,int64,float64,str12,str9,float64,str7,float64,str32,str6,str5,str3
2062,CC1069,17,53.2,4,16,17h57m57.4316s,+04d15m20.4771s,4,3561.0,9.53,M5 V,--,--,-0.7118,0.553,6;38,11.3,--


In [23]:
table3.show_in_notebook()

idx,﻿No.,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D. Number 1,D Number 2,Mag.,Spectral Type,Tot. Mu,Mu alpha,AO Mu alpha,Rel. Parallax,P.E.,Ptg. Mag.,No. Pl
0,780,38 Ceti,1,9.7,-1,31,01h14m06.49s,-00d59m12.3352s,-1,162.0,5.8,F5,0.218,-0.023,-0.04,0.022,7; 24,11.6,21
1,781,Theta Ceti,1,19.0,-8,42,01h24m00.4834s,-08d10m40.857s,-8,244.0,3.8,K0,0.228,-0.077,-95.0,0.031,5; 18,11.9,21
2,782,107 Pisc,1,37.1,19,47,01h42m27.9325s,+20d17m17.7465s,19,279.0,5.3,G5,0.735,-0.292,-0.333,0.125,9; 30,12.1,19
3,783,6 Tria,2,6.6,29,29,02h11m48.0665s,+29d57m15.7867s,29,371.0,5.2,G0,0.088,-0.06,-0.091,-0.001,6; 23,11.8,23
4,784,7 Taur,3,28.5,24,7,03h33m55.3795s,+24d27m13.6185s,23,473.0,5.9,A2,0.034,0.02,-0.015,0.002,5; 17,11.2,21
5,785,Lal 11218,5,51.3,15,44,05h56m45.0406s,+15d44m53.5424s,15,993.0,7.9,G0,0.271,0.086,0.08,0.002,6; 22,10.6,20
6,786,Lal 15743,7,58.8,12,34,08h03m32.8395s,+12d17m11.8303s,12,1759.0,7.9,G5,0.141,0.077,0.102,0.043,5; 15,11.2,18
7,787,14 Boot,14,9.3,13,26,14h13m50.3824s,+12d57m56.0675s,13,2764.0,5.5,F8,0.264,-0.257,-0.211,0.014,8; 27,10.3,20
8,788,Pi 69,14,18.5,8,54,14h22m55.6615s,+08d26m40.2831s,9,2882.0,5.1,A0,0.069,0.065,-0.022,-0.003,7; 23,11.4,21
9,789,Lal 26635,14,31.7,-3,51,14h36m13.2913s,-04d17m10.9087s,-3,3648.0,7.8,G0,0.348,-0.347,-0.248,0.009,7; 22,10.8,19


http://learn.astropy.org/rst-tutorials/conesearch.html?highlight=filtertutorials

In [24]:
ascii.write(table1, 'J2000_table1.csv', format='csv', fast_writer=False) 
ascii.write(table2, 'J2000_table2.csv', format='csv', fast_writer=False) 
ascii.write(table3, 'J2000_table3.csv', format='csv', fast_writer=False) 

# This section is important!
## Query Vizier
Use Vizier to query the YPC online published catalog to find matches between Allegheny published 1900 coordinates and the YPC.
<br>
http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=I/238A/picat&-out.max=50&-out.form=HTML%20Table&-oc.form=sexa

In [25]:
catalog_list = Vizier.find_catalogs('I/238A') # this is the name of the YPC online catalog

In [26]:
Vizier.ROW_LIMIT = -1
catalogs = Vizier.get_catalogs(catalog_list.keys())
print(catalogs)

TableList with 3 tables:
	'0:I/238A/picat' with 15 column(s) and 8994 row(s) 
	'1:I/238A/source' with 3 column(s) and 58 row(s) 
	'2:I/238A/errata' with 3 column(s) and 2132 row(s) 


In [27]:
"""
query_vizier function is no longer relevant because it is encompassed in the second function
this function is useful for querying vizier given ONLY the name of the star
"""

def query_vizier(name):
    
    cols = ['_RAJ2000', '_DEJ2000','RA1900', 'DE1900', 'pm', 'pmPA', 'pi', 'e_pi'] #names of all relevant cols of information, must match actual names of Vizier columns
    v = Vizier(columns=cols)
    search_object = str(name)
    result = v.query_object(search_object,catalog=['I/238A'])
    if result:
        table = result['I/238A/picat']

        ra2000 = result['I/238A/picat']['_RAJ2000']
        dec2000 = result['I/238A/picat']['_DEJ2000']

        table.remove_columns(['_RAJ2000', '_DEJ2000'])


        _RA2000 = Column(np.arange(len(table)), name='_RAJ2000')
        _DE2000 = Column(np.arange(len(table)), name='_DEJ2000')
        table.add_column(_RA2000, index=0)
        table.add_column(_DE2000, index=1)
        table['_RAJ2000'] = table['_RAJ2000'].astype(str)
        table['_DEJ2000'] = table['_DEJ2000'].astype(str)


        c = SkyCoord(ra=ra2000, dec=dec2000)
        radec = c.to_string('hmsdms')[0]
        ra,dec = radec.split()[0],radec.split()[1]


        table['_RAJ2000'] = ra
        table['_DEJ2000'] = dec

        table['_RAJ2000'].unit = "h:m:s"
        table['_DEJ2000'].unit = "d:m:s"
    else: 
        a = 'empty'
        table = Table([[a],[a],[a],[a],[np.nan],[np.nan],[np.nan],[np.nan]], names=('_RAJ2000', '_DEJ2000','RA1900', 'DE1900', 'pm', 'pmPA', 'pi', 'e_pi'))
    
    return table


"""
query_vizier_region is the relevant function
queries the online catalog using both celestial coords and name of the star as published in ALL

"""
    

def query_vizier_region(RAh,RAm,DECd,DECm,name,rad=5):
    """
    Input parameters: J1900 RA hours, RA minutes
                            Dec degrees, Dec minutes
                            name of star
                            radius (in arcminutes) to search within, default = 5"
    """
    
    cols = ['f_GCTP', 'GCTP','comp', 'RA1900','DE1900','Vmag', 'B-V','MK','name', 'pm', 'pmPA', 'pi', 'e_pi','q_pi','_RA.icrs','_DE.icrs']
    v = Vizier(columns=cols)
    

    
    ra_minute = str(RAm)
    ra_minute = ra_minute[:2]
    ra_sec = "%.2f" % ((RAm * 60) % 60)
    
    dec_sec = "%.2f" % ((abs(DECm) * 60) % 60)
 
    dec_d = str(DECd)
    
    if dec_d[0] != '-':
        dec_d = '+' + dec_d
        
    if np.isnan(RAh) == False:
        ra = str(RAh) + ':' + str(ra_minute) + ':' + str(ra_sec) 
        dec = (dec_d) + ':' + str(DECm) + ':' + str(dec_sec) 
        
        radec = ra + ',' +dec
        result = v.query_object(radec, radius=rad*u.arcmin, catalog=['I/238A'],coordinate_frame='B1900')
        
        
        if result:
            table = result['I/238A/picat']
            
        
        else: 
            search_object = str(name).lower()
            result2  = v.query_object(search_object,catalog=['I/238A'])
            
            if result2:
                table = result2['I/238A/picat']
                            
            else:
                newrad = 10
                result3 = v.query_object(radec, radius=newrad*u.arcmin, catalog=['I/238A'],coordinate_frame='B1900')
                
                if result3:
                    table = result3['I/238A/picat']
                
                else:
                    a = '0'
                    table = Table([[bytes(a,encoding='utf8')],[float(0)],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')],[float(0)],[float(0)],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')],[float(0)],[int(0)],[float(0)],[float(0)],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')]], names=('f_GCTP', 'GCTP','comp', 'RA1900','DE1900','Vmag','B-V','MK','name', 'pm', 'pmPA', 'pi', 'e_pi','q_pi','_RA.icrs','_DE.icrs'))
               
    else:
        a = '0'
        table = Table([[bytes(a,encoding='utf8')],[float(0)],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')],[float(0)],[float(0)],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')],[float(0)],[int(0)],[float(0)],[float(0)],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')],[bytes(a,encoding='utf8')]], names=('f_GCTP', 'GCTP','comp', 'RA1900','DE1900','Vmag','B-V','MK','name', 'pm', 'pmPA', 'pi', 'e_pi','q_pi','_RA.icrs','_DE.icrs'))
                
    return table


In [28]:
#query_vizier_region(1,2,54,26,'mu cas') #mu cas
#query_vizier_region(0,43,7,2,'delta piscium')
#query_vizier_region(0,13,36,14,'sig and')

#query_vizier_region(19,3,16,42,'B.D. +16 3752') #required > 5 search radius tcno = 33,idx = 35

#03 38 0.00	+24 16 0.00, nu persei
#query_vizier_region(3,28,24,16,'Nu Persei') #tc 253
#query_vizier_region(2,36,4,59,'Nu Piscium')
query_vizier_region(18,5,2,6,'phi pisc')

#query_vizier_region(0,0,0,0,'al')

f_GCTP,GCTP,comp,RA1900,DE1900,Vmag,B-V,MK,name,pm,pmPA,pi,e_pi,q_pi,_RA.icrs,_DE.icrs
bytes1,float64,bytes1,bytes1,bytes1,float64,float64,bytes1,bytes1,float64,int64,float64,float64,bytes1,bytes1,bytes1
0,0.0,0,0,0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0


In [29]:
query_vizier('adaf')

_RAJ2000,_DEJ2000,RA1900,DE1900,pm,pmPA,pi,e_pi
str5,str5,str5,str5,float64,float64,float64,float64
empty,empty,empty,empty,nan,nan,nan,nan


In [30]:
t4 = query_vizier('Sig And')
print(t4.colnames)

['_RAJ2000', '_DEJ2000', 'RA1900', 'DE1900', 'pm', 'pmPA', 'pi', 'e_pi']


### Now, run through all three tables to search for YPC match and append matching information to existing table

In [31]:
"""
table 1 
"""
newrows_t1 = []

for i in range(len(table1)):
    print(i)
    rah = table1['RA 1900 (Hour)'][i]
    ram = table1['RA 1900 (Min)'][i]
    decd = table1['Dec 1900 (Degrees)'][i]
    decm = table1['Dec 1900 (Min)'][i]
    searchname = table1['Name'][i]
    t2_1 = query_vizier_region(rah,ram,decd,decm,searchname)
    newrows_t1.append(hstack([table1[i], t2_1]))
    




    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


...

304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


344
345
346
347
348
349
350
351
352
353
354
355


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381


In [32]:
table1[0]

﻿Number,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D Number 1,D Number 2,Mag.,Spectral type,Prop. Mot.,Rel. Parallax,Error,Error for one good plate
int64,str25,int64,int64,int64,int64,str21,str21,int64,int64,str4,str3,float64,str6,str32,str32
1,Sigma Andromedae,0,13,36,14,00h18m14.1243s,+36d47m19.9256s,35,44,4.5,A2,0.08,0.012,0.008,0.022


In [33]:
newtable1 = vstack(newrows_t1)

In [34]:
newtable1.show_in_notebook()

idx,﻿Number,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D Number 1,D Number 2,Mag.,Spectral type,Prop. Mot.,Rel. Parallax,Error,Error for one good plate,f_GCTP,GCTP,comp,RA1900,DE1900,Vmag,B-V,MK,name,pm,pmPA,pi,e_pi,q_pi,_RA.icrs,_DE.icrs
,,,,,,,,,,,,,,,,,,,,"""h:m:s""","""d:m:s""",mag,mag,,,arcsec / yr,deg,arcsec,mas,,"""h:m:s""","""d:m:s"""
0,1,Sigma Andromedae,0,13,36,14,00h18m14.1243s,+36d47m19.9256s,35,44,4.5,A2,0.08,0.012,0.008,0.022,,51.00,,00 13 06.0,+36 13 51,4.52,0.05,A2 V,SIG And,0.074,240,0.0193,12.5,,00 18 19.7,+36 47 07
1,2,Delta Piscium,0,43,7,2,00h48m10.7732s,+07d34m45.0455s,6,107,4.5,K5,0.09,0.012,7.0,20.0,*,158.00,,00 43 29.5,+07 02 27,4.43,1.50,K4 IIIb,DEL Psc,0.095,119,0.0176,9.4,G,00 48 40.9,+07 35 06
2,3,Epsilon Piscium,0,58,7,21,01h03m12.059s,+07d53m14.9045s,7,153,4.5,G5,0.08,0.023,8.0,23.0,,209.00,,00 57 45.0,+07 21 06,4.28,0.96,K0 III,EPS Psc,0.085,289,0.0261,6.5,G,01 02 56.5,+07 53 24
3,4,Mu Cassiopeiae,1,2,54,26,01h08m00.2082s,+54d58m04.4907s,54,223,5.3,G5,3.75,0.143,9.0,23.0,*,219.00,A,01 01 36.7,+54 25 47,5.17,0.69,G5 Vb,MU Cas,3.770,115,0.1339,1.7,G,01 08 16.2,+54 55 11
4,5,Omicron Piscium,1,40,8,39,01h45m16.4502s,+09d09m06.8914s,8,273,4.5,G5,0.09,0.012,6.0,18.0,,368.00,,01 40 06.6,+08 39 16,4.26,0.96,G8 III,OMI Psc,0.089,53,0.0216,7.7,G,01 45 23.6,+09 09 28
5,6,Beta Arietis,1,49,20,19,01h54m30.8763s,+20d48m30.9278s,20,306,2.7,A5,0.15,0.072,8.0,19.0,,394.00,,01 49 06.8,+20 19 09,2.64,0.13,A5 V,BET Ari,0.144,138,0.0671,7.8,F,01 54 38.4,+20 48 28
6,7,39 Arietis,2,42,28,50,02h47m56.3088s,+29d15m07.2932s,28,462,4.6,K,0.18,0.022,6.0,16.0,,567.00,,02 41 57.0,+28 49 55,4.51,1.11,K1.5 III,39 Ari,0.207,130,0.0204,1.9,G,02 47 54.5,+29 14 49
7,8,37 Tauri,3,59,21,49,04h04m54.2667s,+22d05m27.4542s,21,585,4.5,K,0.11,-0.022,9.0,24.0,*,897.00,A,03 58 46.8,+21 48 31,4.36,1.07,K0+ III-*,37 Tau,0.107,122,0.0125,5.7,F,04 04 41.7,+22 04 54
8,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,*,897.00,B,03 58 45.3,+21 46 16,10.40,--,G,,0.020,172,--,--,,04 04 39.4,+22 02 43


In [35]:
number = 0

for i in range(len(newtable1)):
    if newtable1[i]['_RA.icrs'] == 'empty':
        number += 1
        

In [36]:
print(number/len(newtable1))
print(number, len(newtable1))

0.0
0 486


In [37]:
"""
table 2 
"""
"""
newrows_t2 = []

for i in range(len(table2)):
    print(i)
    t2_2 = query_vizier(table2['Name'][i])
    newrows_t2.append(hstack([table2[i], t2_2]))
"""
    
newrows_t2 = []

for i in range(len(table2)):
    print(i)
    rah = table2['RA 1900 (Hour)'][i]
    ram = table2['RA 1900 (Min)'][i]
    decd = table2['Dec 1900 (Degrees)'][i]
    decm = table2['Dec 1900 (Min)'][i]
    searchname = table2['Name'][i]
    t2_2 = query_vizier_region(rah,ram,decd,decm,searchname)
    newrows_t2.append(hstack([table2[i], t2_2]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


61
62
63
64
65
66
67
68
69
70
71
72


...

385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


440
441
442
443
444
445


In [38]:
"""
table 3
"""
"""
newrows_t3 = []

for i in range(len(table1)):
    print(i)
    t2_3 = query_vizier(table3['Name'][i])
    newrows_t3.append(hstack([table3[i], t2_3]))
"""

newrows_t3 = []

for i in range(len(table3)):
    print(i)
    rah = table3['RA 1900 (Hour)'][i]
    ram = table3['RA 1900 (Min)'][i]
    decd = table3['Dec 1900 (Degrees)'][i]
    decm = table3['Dec 1900 (Min)'][i]
    searchname = table3['Name'][i]
    t2_3 = query_vizier_region(rah,ram,decd,decm,searchname)
    newrows_t3.append(hstack([table3[i], t2_3]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


108
109
110
111
112
113
114


...

1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Warning: converting a masked element to nan.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Warning: converting a masked element to nan.


1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319


In [39]:
newtable2 = vstack(newrows_t2)
newtable3 = vstack(newrows_t3)

In [40]:
#display all of them to check for matches and any formatting thay may need changed
newtable1.show_in_notebook()

idx,﻿Number,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D Number 1,D Number 2,Mag.,Spectral type,Prop. Mot.,Rel. Parallax,Error,Error for one good plate,f_GCTP,GCTP,comp,RA1900,DE1900,Vmag,B-V,MK,name,pm,pmPA,pi,e_pi,q_pi,_RA.icrs,_DE.icrs
,,,,,,,,,,,,,,,,,,,,"""h:m:s""","""d:m:s""",mag,mag,,,arcsec / yr,deg,arcsec,mas,,"""h:m:s""","""d:m:s"""
0,1,Sigma Andromedae,0,13,36,14,00h18m14.1243s,+36d47m19.9256s,35,44,4.5,A2,0.08,0.012,0.008,0.022,,51.00,,00 13 06.0,+36 13 51,4.52,0.05,A2 V,SIG And,0.074,240,0.0193,12.5,,00 18 19.7,+36 47 07
1,2,Delta Piscium,0,43,7,2,00h48m10.7732s,+07d34m45.0455s,6,107,4.5,K5,0.09,0.012,7.0,20.0,*,158.00,,00 43 29.5,+07 02 27,4.43,1.50,K4 IIIb,DEL Psc,0.095,119,0.0176,9.4,G,00 48 40.9,+07 35 06
2,3,Epsilon Piscium,0,58,7,21,01h03m12.059s,+07d53m14.9045s,7,153,4.5,G5,0.08,0.023,8.0,23.0,,209.00,,00 57 45.0,+07 21 06,4.28,0.96,K0 III,EPS Psc,0.085,289,0.0261,6.5,G,01 02 56.5,+07 53 24
3,4,Mu Cassiopeiae,1,2,54,26,01h08m00.2082s,+54d58m04.4907s,54,223,5.3,G5,3.75,0.143,9.0,23.0,*,219.00,A,01 01 36.7,+54 25 47,5.17,0.69,G5 Vb,MU Cas,3.770,115,0.1339,1.7,G,01 08 16.2,+54 55 11
4,5,Omicron Piscium,1,40,8,39,01h45m16.4502s,+09d09m06.8914s,8,273,4.5,G5,0.09,0.012,6.0,18.0,,368.00,,01 40 06.6,+08 39 16,4.26,0.96,G8 III,OMI Psc,0.089,53,0.0216,7.7,G,01 45 23.6,+09 09 28
5,6,Beta Arietis,1,49,20,19,01h54m30.8763s,+20d48m30.9278s,20,306,2.7,A5,0.15,0.072,8.0,19.0,,394.00,,01 49 06.8,+20 19 09,2.64,0.13,A5 V,BET Ari,0.144,138,0.0671,7.8,F,01 54 38.4,+20 48 28
6,7,39 Arietis,2,42,28,50,02h47m56.3088s,+29d15m07.2932s,28,462,4.6,K,0.18,0.022,6.0,16.0,,567.00,,02 41 57.0,+28 49 55,4.51,1.11,K1.5 III,39 Ari,0.207,130,0.0204,1.9,G,02 47 54.5,+29 14 49
7,8,37 Tauri,3,59,21,49,04h04m54.2667s,+22d05m27.4542s,21,585,4.5,K,0.11,-0.022,9.0,24.0,*,897.00,A,03 58 46.8,+21 48 31,4.36,1.07,K0+ III-*,37 Tau,0.107,122,0.0125,5.7,F,04 04 41.7,+22 04 54
8,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,*,897.00,B,03 58 45.3,+21 46 16,10.40,--,G,,0.020,172,--,--,,04 04 39.4,+22 02 43


In [41]:
newtable2.show_in_notebook()

idx,﻿Number,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D Number 1,D Number 2,Mag.,Spectral Type,P.M. Catalog,P.M. AO,Rel. Parallax,P.E,P.E. Wt,sqrt(p pi),No. Plates,f_GCTP,GCTP,comp,RA1900,DE1900,Vmag,B-V,MK,name,pm,pmPA,pi,e_pi,q_pi,_RA.icrs,_DE.icrs
,,,,,,,,,,,,,,,,,,,,,,,"""h:m:s""","""d:m:s""",mag,mag,,,arcsec / yr,deg,arcsec,mas,,"""h:m:s""","""d:m:s"""
0,366,22 Andr,0,5.1,45,31,00h10m11.9438s,+46d04m23.477s,45,17,5.1,FO,0.006,0.011,-0.01,0.01,0.025,2.5,14,,20.00,,00 05 07.1,+45 30 56,5.03,0.40,F2 II,22 And,0.008,48,-0.0052,15.4,,00 10 19.2,+46 04 19
1,367,Lal 617,0,23.2,9,39,00h28m09.9926s,+10d12m12.1009s,9,47,6,F2,0.031*,0.031,0.029,7.0,21.0,3.0,17,,76.00,,00 23 09.7,+09 38 32,6.04,0.43,F6 Va vw,,0.206,168,0.0329,11.1,,00 28 20.0,+10 11 23
2,368,Phi Pisc,1,8.3,24,3,01h13m25.7247s,+24d34m49.9085s,23,158,4.6,KO,0.025,-0.032,-0.01,8.0,23.0,2.9,15,0,0.00,0,0,0,0.00,0.00,0,0,0.000,0,0.0000,0.0,0,0,0
3,369,Omega Andr.,1,21.7,44,53,01h26m55.5002s,+45d24m11.5507s,44,307,5,F5,0.346,0.337,0.024,4.0,13.0,3.0,15,,298.00,,01 21 40.1,+44 53 25,4.83,0.42,F5 IV,OME And,0.365,106,0.0270,6.3,G,01 27 39.3,+45 24 24
4,370,Piazzi 145,1,35.7,25,14,01h40m33.7504s,+25d44m24.8766s,25,276,6.3,F5,0.13,0.106,0.009,10.0,24.0,2.5,14,,351.00,,01 35 43.5,+25 14 26,6.17,0.44,F2 III,,0.124,106,0.0259,10.1,G,01 41 18.3,+25 44 44
5,371,Lal 4268,2,12.8,1,17,02h17m09.3454s,+01d44m48.7873s,1,410,5.8,F8,0.371,0.4,0.043,10.0,25.0,2.6,12,,473.00,,02 12 49.6,+01 17 06,5.58,0.60,G0.5 IVb,Lal 4268,0.522,44,0.0359,8.7,G,02 18 01.4,+01 45 28
6,372,Lal 5376,2,49.7,26,28,02h54m53.1559s,+26d52m26.3894s,26,484,7.4,G5,0.258*,0.269,0.056,7.0,20.0,2.8,14,,603.00,M,02 49 43.7,+26 28 20,7.58,0.92,K2 d,SIG 326A,0.324,124,0.0393,5.9,F,02 55 39.0,+26 52 23
7,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,,603.00,A,02 49 43.7,+26 28 20,7.62,0.93,K2 d,SIG 326A,0.324,124,0.0393,5.9,F,02 55 39.0,+26 52 23
8,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,*,603.00,B,02 49 44.0,+26 28 20,9.80,1.40,M0 d,SIG 326B,0.322,125,--,--,,02 55 39.3,+26 52 23


In [42]:
newtable3.show_in_notebook()

idx,﻿No.,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D. Number 1,D Number 2,Mag.,Spectral Type,Tot. Mu,Mu alpha,AO Mu alpha,Rel. Parallax,P.E.,Ptg. Mag.,No. Pl,f_GCTP,GCTP,comp,RA1900,DE1900,Vmag,B-V,MK,name,pm,pmPA,pi,e_pi,q_pi,_RA.icrs,_DE.icrs
,,,,,,,,,,,,,,,,,,,,,,,"""h:m:s""","""d:m:s""",mag,mag,,,arcsec / yr,deg,arcsec,mas,,"""h:m:s""","""d:m:s"""
0,780,38 Ceti,1,9.7,-1,31,01h14m06.49s,-00d59m12.3352s,-1,162.0,5.8,F5,0.218,-0.023,-0.04,0.022,7; 24,11.6,21,*,254.00,,01 09 42.7,-01 30 31,5.70,0.42,F5 V,38 Cet,0.207,356,0.0279,7.7,G,01 14 49.1,-00 58 25
1,781,Theta Ceti,1,19.0,-8,42,01h24m00.4834s,-08d10m40.857s,-8,244.0,3.8,K0,0.228,-0.077,-95.0,0.031,5; 18,11.9,21,,286.00,,01 19 01.4,-08 41 57,3.60,1.06,K0 III-*,THE Cet,0.231,201,0.0355,2.6,G,01 24 01.3,-08 11 00
2,782,107 Pisc,1,37.1,19,47,01h42m27.9325s,+20d17m17.7465s,19,279.0,5.3,G5,0.735,-0.292,-0.333,0.125,9; 30,12.1,19,*,356.00,,01 37 03.9,+19 46 57,5.24,0.84,K1 V,107 Psc,0.749,203,0.1326,2.3,G,01 42 29.8,+20 16 05
3,783,6 Tria,2,6.6,29,29,02h11m48.0665s,+29d57m15.7867s,29,371.0,5.2,G0,0.088,-0.06,-0.091,-0.001,6; 23,11.8,23,,447.00,A,02 06 33.9,+29 50 04,4.94,0.78,"G5+F5III,V",6 Tri,0.086,220,0.0023,9.8,,02 12 22.3,+30 18 10
4,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,,447.00,B,02 06 36.0,+29 50 00,6.60,--,F6 V,,--,--,--,--,,02 12 24.8,+30 18 13
5,784,7 Taur,3,28.5,24,7,03h33m55.3795s,+24d27m13.6185s,23,473.0,5.9,A2,0.034,0.02,-0.015,0.002,5; 17,11.2,21,,744.00,A,03 28 31.0,+24 07 44,5.92,0.13,"A3+A3V,V",SIG 412AB,0.024,166,0.0048,6.9,G,03 34 26.5,+24 27 51
6,785,Lal 11218,5,51.3,15,44,05h56m45.0406s,+15d44m53.5424s,15,993.0,7.9,G0,0.271,0.086,0.08,0.002,6; 22,10.6,20,*,1368.00,,05 51 16.1,+15 44 02,8.21,0.64,G4 d,,0.258,161,0.0049,9.8,,05 57 01.7,+15 44 29
7,786,Lal 15743,7,58.8,12,34,08h03m32.8395s,+12d17m11.8303s,12,1759.0,7.9,G5,0.141,0.077,0.102,0.043,5; 15,11.2,18,*,1916.00,M,07 58 49.6,+12 34 31,7.78,0.85,K0 V,,0.177,147,0.0329,3.8,G,08 04 23.1,+12 17 22
8,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,*,1916.00,B,07 58 48.0,+12 35 00,8.80,--,,,--,--,--,--,,08 04 20.8,+12 18 06


In [43]:
newtable1['GCTP'].info.format = '.2f'
newtable2['GCTP'].info.format = '.2f'
newtable3['GCTP'].info.format = '.2f'

In [44]:
### save all three tables as new .csv files

ascii.write(newtable1, 'newtable1.csv', format='csv',overwrite=True)  
ascii.write(newtable2, 'newtable2.csv', format='csv',overwrite=True)
ascii.write(newtable3, 'newtable3.csv', format='csv',overwrite=True)

In [45]:

newtable1

﻿Number,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D Number 1,D Number 2,Mag.,Spectral type,Prop. Mot.,Rel. Parallax,Error,Error for one good plate,f_GCTP,GCTP,comp,RA1900,DE1900,Vmag,B-V,MK,name,pm,pmPA,pi,e_pi,q_pi,_RA.icrs,_DE.icrs
,,,,,,,,,,,,,,,,,,,"""h:m:s""","""d:m:s""",mag,mag,,,arcsec / yr,deg,arcsec,mas,,"""h:m:s""","""d:m:s"""
int64,str25,int64,int64,int64,int64,str21,str21,int64,int64,str4,str3,float64,str6,str32,str32,bytes1,float64,bytes1,bytes10,bytes9,float64,float64,bytes10,bytes11,float64,int64,float64,float64,bytes1,bytes10,bytes9
1,Sigma Andromedae,0,13,36,14,00h18m14.1243s,+36d47m19.9256s,35,44,4.5,A2,0.08,0.012,0.008,0.022,,51.00,,00 13 06.0,+36 13 51,4.52,0.05,A2 V,SIG And,0.074,240,0.0193,12.5,,00 18 19.7,+36 47 07
2,Delta Piscium,0,43,7,2,00h48m10.7732s,+07d34m45.0455s,6,107,4.5,K5,0.09,0.012,7.0,20.0,*,158.00,,00 43 29.5,+07 02 27,4.43,1.50,K4 IIIb,DEL Psc,0.095,119,0.0176,9.4,G,00 48 40.9,+07 35 06
3,Epsilon Piscium,0,58,7,21,01h03m12.059s,+07d53m14.9045s,7,153,4.5,G5,0.08,0.023,8.0,23.0,,209.00,,00 57 45.0,+07 21 06,4.28,0.96,K0 III,EPS Psc,0.085,289,0.0261,6.5,G,01 02 56.5,+07 53 24
4,Mu Cassiopeiae,1,2,54,26,01h08m00.2082s,+54d58m04.4907s,54,223,5.3,G5,3.75,0.143,9.0,23.0,*,219.00,A,01 01 36.7,+54 25 47,5.17,0.69,G5 Vb,MU Cas,3.770,115,0.1339,1.7,G,01 08 16.2,+54 55 11
5,Omicron Piscium,1,40,8,39,01h45m16.4502s,+09d09m06.8914s,8,273,4.5,G5,0.09,0.012,6.0,18.0,,368.00,,01 40 06.6,+08 39 16,4.26,0.96,G8 III,OMI Psc,0.089,53,0.0216,7.7,G,01 45 23.6,+09 09 28
6,Beta Arietis,1,49,20,19,01h54m30.8763s,+20d48m30.9278s,20,306,2.7,A5,0.15,0.072,8.0,19.0,,394.00,,01 49 06.8,+20 19 09,2.64,0.13,A5 V,BET Ari,0.144,138,0.0671,7.8,F,01 54 38.4,+20 48 28
7,39 Arietis,2,42,28,50,02h47m56.3088s,+29d15m07.2932s,28,462,4.6,K,0.18,0.022,6.0,16.0,,567.00,,02 41 57.0,+28 49 55,4.51,1.11,K1.5 III,39 Ari,0.207,130,0.0204,1.9,G,02 47 54.5,+29 14 49
8,37 Tauri,3,59,21,49,04h04m54.2667s,+22d05m27.4542s,21,585,4.5,K,0.11,-0.022,9.0,24.0,*,897.00,A,03 58 46.8,+21 48 31,4.36,1.07,K0+ III-*,37 Tau,0.107,122,0.0125,5.7,F,04 04 41.7,+22 04 54


In [46]:
newtable2

﻿Number,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D Number 1,D Number 2,Mag.,Spectral Type,P.M. Catalog,P.M. AO,Rel. Parallax,P.E,P.E. Wt,sqrt(p pi),No. Plates,f_GCTP,GCTP,comp,RA1900,DE1900,Vmag,B-V,MK,name,pm,pmPA,pi,e_pi,q_pi,_RA.icrs,_DE.icrs
,,,,,,,,,,,,,,,,,,,,,,"""h:m:s""","""d:m:s""",mag,mag,,,arcsec / yr,deg,arcsec,mas,,"""h:m:s""","""d:m:s"""
int64,str22,int64,float64,int64,int64,str21,str21,int64,int64,str4,str4,str7,str6,str6,str32,str32,float64,int64,bytes1,float64,bytes1,bytes10,bytes9,float64,float64,bytes10,bytes11,float64,int64,float64,float64,bytes1,bytes10,bytes9
366,22 Andr,0,5.1,45,31,00h10m11.9438s,+46d04m23.477s,45,17,5.1,FO,0.006,0.011,-0.01,0.01,0.025,2.5,14,,20.00,,00 05 07.1,+45 30 56,5.03,0.40,F2 II,22 And,0.008,48,-0.0052,15.4,,00 10 19.2,+46 04 19
367,Lal 617,0,23.2,9,39,00h28m09.9926s,+10d12m12.1009s,9,47,6,F2,0.031*,0.031,0.029,7.0,21.0,3.0,17,,76.00,,00 23 09.7,+09 38 32,6.04,0.43,F6 Va vw,,0.206,168,0.0329,11.1,,00 28 20.0,+10 11 23
368,Phi Pisc,1,8.3,24,3,01h13m25.7247s,+24d34m49.9085s,23,158,4.6,KO,0.025,-0.032,-0.01,8.0,23.0,2.9,15,0,0.00,0,0,0,0.00,0.00,0,0,0.000,0,0.0000,0.0,0,0,0
369,Omega Andr.,1,21.7,44,53,01h26m55.5002s,+45d24m11.5507s,44,307,5,F5,0.346,0.337,0.024,4.0,13.0,3.0,15,,298.00,,01 21 40.1,+44 53 25,4.83,0.42,F5 IV,OME And,0.365,106,0.0270,6.3,G,01 27 39.3,+45 24 24
370,Piazzi 145,1,35.7,25,14,01h40m33.7504s,+25d44m24.8766s,25,276,6.3,F5,0.13,0.106,0.009,10.0,24.0,2.5,14,,351.00,,01 35 43.5,+25 14 26,6.17,0.44,F2 III,,0.124,106,0.0259,10.1,G,01 41 18.3,+25 44 44
371,Lal 4268,2,12.8,1,17,02h17m09.3454s,+01d44m48.7873s,1,410,5.8,F8,0.371,0.4,0.043,10.0,25.0,2.6,12,,473.00,,02 12 49.6,+01 17 06,5.58,0.60,G0.5 IVb,Lal 4268,0.522,44,0.0359,8.7,G,02 18 01.4,+01 45 28
372,Lal 5376,2,49.7,26,28,02h54m53.1559s,+26d52m26.3894s,26,484,7.4,G5,0.258*,0.269,0.056,7.0,20.0,2.8,14,,603.00,M,02 49 43.7,+26 28 20,7.58,0.92,K2 d,SIG 326A,0.324,124,0.0393,5.9,F,02 55 39.0,+26 52 23
--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,,603.00,A,02 49 43.7,+26 28 20,7.62,0.93,K2 d,SIG 326A,0.324,124,0.0393,5.9,F,02 55 39.0,+26 52 23


In [47]:
newtable3

﻿No.,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D. Number 1,D Number 2,Mag.,Spectral Type,Tot. Mu,Mu alpha,AO Mu alpha,Rel. Parallax,P.E.,Ptg. Mag.,No. Pl,f_GCTP,GCTP,comp,RA1900,DE1900,Vmag,B-V,MK,name,pm,pmPA,pi,e_pi,q_pi,_RA.icrs,_DE.icrs
,,,,,,,,,,,,,,,,,,,,,,"""h:m:s""","""d:m:s""",mag,mag,,,arcsec / yr,deg,arcsec,mas,,"""h:m:s""","""d:m:s"""
str5,str44,int64,float64,int64,int64,str21,str21,int64,float64,str12,str9,float64,str7,float64,str32,str6,str5,str3,bytes1,float64,bytes1,bytes10,bytes9,float64,float64,bytes10,bytes11,float64,int64,float64,float64,bytes1,bytes10,bytes9
780,38 Ceti,1,9.7,-1,31,01h14m06.49s,-00d59m12.3352s,-1,162.0,5.8,F5,0.218,-0.023,-0.04,0.022,7; 24,11.6,21,*,254.00,,01 09 42.7,-01 30 31,5.70,0.42,F5 V,38 Cet,0.207,356,0.0279,7.7,G,01 14 49.1,-00 58 25
781,Theta Ceti,1,19.0,-8,42,01h24m00.4834s,-08d10m40.857s,-8,244.0,3.8,K0,0.228,-0.077,-95.0,0.031,5; 18,11.9,21,,286.00,,01 19 01.4,-08 41 57,3.60,1.06,K0 III-*,THE Cet,0.231,201,0.0355,2.6,G,01 24 01.3,-08 11 00
782,107 Pisc,1,37.1,19,47,01h42m27.9325s,+20d17m17.7465s,19,279.0,5.3,G5,0.735,-0.292,-0.333,0.125,9; 30,12.1,19,*,356.00,,01 37 03.9,+19 46 57,5.24,0.84,K1 V,107 Psc,0.749,203,0.1326,2.3,G,01 42 29.8,+20 16 05
783,6 Tria,2,6.6,29,29,02h11m48.0665s,+29d57m15.7867s,29,371.0,5.2,G0,0.088,-0.06,-0.091,-0.001,6; 23,11.8,23,,447.00,A,02 06 33.9,+29 50 04,4.94,0.78,"G5+F5III,V",6 Tri,0.086,220,0.0023,9.8,,02 12 22.3,+30 18 10
--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,,447.00,B,02 06 36.0,+29 50 00,6.60,--,F6 V,,--,--,--,--,,02 12 24.8,+30 18 13
784,7 Taur,3,28.5,24,7,03h33m55.3795s,+24d27m13.6185s,23,473.0,5.9,A2,0.034,0.02,-0.015,0.002,5; 17,11.2,21,,744.00,A,03 28 31.0,+24 07 44,5.92,0.13,"A3+A3V,V",SIG 412AB,0.024,166,0.0048,6.9,G,03 34 26.5,+24 27 51
785,Lal 11218,5,51.3,15,44,05h56m45.0406s,+15d44m53.5424s,15,993.0,7.9,G0,0.271,0.086,0.08,0.002,6; 22,10.6,20,*,1368.00,,05 51 16.1,+15 44 02,8.21,0.64,G4 d,,0.258,161,0.0049,9.8,,05 57 01.7,+15 44 29
786,Lal 15743,7,58.8,12,34,08h03m32.8395s,+12d17m11.8303s,12,1759.0,7.9,G5,0.141,0.077,0.102,0.043,5; 15,11.2,18,*,1916.00,M,07 58 49.6,+12 34 31,7.78,0.85,K0 V,,0.177,147,0.0329,3.8,G,08 04 23.1,+12 17 22


﻿Number,Name,RA 1900 (Hour),RA 1900 (Min),Dec 1900 (Degrees),Dec 1900 (Min),RA 2000,Dec 2000,D Number 1,D Number 2,Mag.,Spectral type,Prop. Mot.,Rel. Parallax,Error,Error for one good plate,f_GCTP,GCTP,comp,RA1900,DE1900,Vmag,B-V,MK,name,pm,pmPA,pi,e_pi,q_pi,_RA.icrs,_DE.icrs
,,,,,,,,,,,,,,,,,,,"""h:m:s""","""d:m:s""",mag,mag,,,arcsec / yr,deg,arcsec,mas,,"""h:m:s""","""d:m:s"""
int64,str25,int64,int64,int64,int64,str21,str21,int64,int64,str4,str3,float64,str6,str32,str32,bytes1,float64,bytes1,bytes10,bytes9,float64,float64,bytes10,bytes11,float64,int64,float64,float64,bytes1,bytes10,bytes9
61,Sigma Geminorum,7,37,29,8,07h43m14.5503s,+28d53m51.8392s,29,1590,4.3,K0,0.24,0.015,6.0,13.0,0,0.00,0,0,0,0.00,0.00,0,0,0.000,0,0.0000,0.0,0,0,0
